In [20]:
import duckdb


In [21]:
con = duckdb.connect(database='dados_duckdb.db', read_only=False)

In [22]:
df = con.execute("""
                SELECT *
                FROM (
                    SELECT *, ROW_NUMBER() OVER(PARTITION BY NATBR ORDER BY data_ingestao DESC) AS row
                    FROM bronze_z0019
                    WHERE data_ingestao >= '2025-03-08'
                ) AS t
                WHERE row = 1
                """).fetchdf()
df.head()

,NATBR,MAKTX,WERKS,MAINS,LABST,nome_arquivo,data_ingestao,row
0,1025,CIMENTO 50KG,BT10,"10,00",100,z0019_2.csv,2025-03-09 09:08:38.858491,1
1,1068,LUVA DE PROTEÇÃO,BT20,"20,00",200,z0019_2.csv,2025-03-09 09:08:38.858491,1
2,1070,MÁSCARA DESCARTÁVEL,BT20,"20,00",200,z0019_2.csv,2025-03-09 09:08:38.858491,1
3,1082,COLHER DE PEDREIRO,BT20,"20,00",200,z0019_2.csv,2025-03-09 09:08:38.858491,1
4,1015,TRENA 5M,BT10,"10,00",100,z0019_2.csv,2025-03-09 09:08:38.858491,1


In [23]:
df_final = df.drop(columns=['nome_arquivo', 'data_ingestao', 'row'])
df_final = df_final.rename(columns={"NATBR": "id "})
df_final = df_final.rename(columns={"MAKTX":"nm_produto"})
df_final = df_final.rename(columns={"WERKS":"id_categoria"})
df_final = df_final.rename(columns={"MAINS":"id_fornecedor"})
df_final = df_final.rename(columns={"LABST": "vl_preco"})
df_final.head(10)
#df_final.dtypes

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,1025,CIMENTO 50KG,BT10,"10,00",100
1,1068,LUVA DE PROTEÇÃO,BT20,"20,00",200
2,1070,MÁSCARA DESCARTÁVEL,BT20,"20,00",200
3,1082,COLHER DE PEDREIRO,BT20,"20,00",200
4,1015,TRENA 5M,BT10,"10,00",100
5,1017,TUBO PVC 40MM,BT10,"10,00",100
6,1032,TIJOLO 6 FUROS,BT20,"20,00",200
7,1039,REJUNTE BRANCO,BT10,"10,00",100
8,1058,FITA CREPE,BT20,"20,00",200
9,1114,SERRA TICO-TICO,BT20,"20,00",200


In [24]:
# Verifica as colunas existentes no DataFrame
colunas_existentes = df_final.columns

# Filtra apenas as colunas que existem no DataFrame para evitar erros
tipos = {
    'id': int,
    'nm_produto': str,
    'id_categoria': str,
    'id_fornecedor': float,
    'vl_preco': float,
}

tipos_filtrados = {col: tipo for col, tipo in tipos.items() if col in colunas_existentes}

# Aplica a conversão de tipos apenas nas colunas existentes
df2 = df_final.astype(tipos_filtrados)

ValueError: could not convert string to float: '10,00': Error while type casting for column 'id_fornecedor'